### Predicting Illiquid Stock Prices using data 

1. Open Anaconda prompt
2. cd into project folder
3. conda env create --file environment.yaml
4. conda activate berkeley_env



In [ ]:
import pandas as pd
import numpy as np
import os 
import glob
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import shap

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

### Load and Format Data


In [ ]:
# financial ratios data for energy companies
# @S&P Global
ratio = pd.read_csv("fundamental_data/energy_ratios.csv")
# historical stock prices - 
# @Yahoo Finance API via yfinance Python package
prices_full = pd.read_csv('stock_prices/prices_y_finance_full.csv')


In [ ]:
# historical ETF data for top market indexes
# @https://www.kaggle.com/datasets/borismarjanovic/price-volume-data-for-all-us-stocks-etfs
mkt_files = glob.glob("market_data/*.txt")
mkt_indexes = pd.concat((pd.read_csv(f).tail(1000).assign(Ticker = f.replace("market_data/", '')
                                                          .replace('.us.txt', '')) for f in mkt_files), 
                                                           ignore_index=True)

In [ ]:
mkt_indexes['Date'] = pd.to_datetime(mkt_indexes['Date']) 

In [ ]:
prices_full['Date'] = pd.to_datetime(prices_full['Date'], utc=True)
prices_full['Date'] = prices_full['Date'].dt.floor('D')
prices_full['Date']  = prices_full['Date'].dt.tz_localize(None)
prices_full = prices_full.rename(columns={'Close':'price'})

In [ ]:
prices_cols = ['Date', 'Ticker', 'price']

In [ ]:
ratio = ratio.rename(columns={'EXCHANGE TICKER SYMBOL - HISTORICAL': 'Ticker'})
ratio['Public Date'] = pd.to_datetime(ratio['Public Date'].astype('str'))
ratio = ratio.drop(columns=['Unnamed: 0'])
ratio = ratio.rename(columns={'Public Date':'quarter_date'})

In [ ]:
ratio_dates = ratio.quarter_date.unique()

#### Inspect financial ratio data

In [ ]:
ratio.head()

In [ ]:
def filter_max_dates(row):
    return max(ratio_dates[np.where(row['Date'] > ratio_dates)])
    

#### Since financial ratio data is quarterly and market index data is daily, we want to match them up appropriately

In [ ]:
mkt_indexes['quarter_date'] = mkt_indexes.apply(filter_max_dates, axis=1)

In [ ]:
mkt_indexes = pd.pivot_table(mkt_indexes[['Date', 'Close', 'Ticker', 'quarter_date']], values='Close', columns=['Ticker'],
              index=['Date', 'quarter_date']).reset_index()

In [ ]:
financial = pd.merge(mkt_indexes, ratio, how='left', on=['quarter_date'])


In [ ]:
financial.head()

#### Inspect a single stock's price over time

In [ ]:
ticker = 'APA'
plt.plot(prices_full[prices_full.Ticker == ticker].Date,prices_full[prices_full.Ticker == ticker].price)
plt.title('Historical Stock Price for {}'.format(ticker))
plt.xticks(rotation=45)
plt.show()


In [ ]:
full_dataset = pd.merge(financial, prices_full[prices_cols], how='left', on=['Ticker', 'Date']).dropna(subset=['price'])


#### Inspect data types and missing values

In [ ]:
full_dataset.info()

#### Drop sparse features

In [ ]:
full_dataset = full_dataset.drop(columns=['Trailing P/E to Growth (PEG) ratio', 'Dividend Yield'])

In [ ]:
full_dataset = full_dataset.dropna()
full_dataset = full_dataset.sort_values(by='Date', ascending=True)
full_dataset.shape

In [ ]:
full_dataset.head()

#### Filter out outlier stock prices

In [ ]:
full_dataset = full_dataset[full_dataset.price < 1000]

In [ ]:
mkt_features = ['qqq', 'spy', 'vgsh', 'vxx']

In [ ]:
fundamental_features = ['Enterprise Value Multiple',
       'P/E (Diluted, Excl. EI)', 'Price/Cash flow', 'Net Profit Margin',
       'Operating Profit Margin Before Depreciation', 'Cash Flow Margin',
       'Total Debt/Invested Capital', 'Interest/Average Total Debt',
       'Cash Balance/Total Liabilities', 'Total Debt/EBITDA',
       'Profit Before Depreciation/Current Liabilities',
       'Operating CF/Current Liabilities', 'Cash Flow/Total Debt',
       'Free Cash Flow/Operating Cash Flow',
       'Total Liabilities/Total Tangible Assets', 'Total Debt/Capital',
       'Total Debt/Equity', 'Cash Ratio', 'Quick Ratio (Acid Test)',
       'Price/Book']

In [ ]:
features = mkt_features + fundamental_features


In [ ]:
target = 'price'

### EDA

#### Analyze market indexes

In [ ]:
fig, ax1 = plt.subplots(figsize=(7, 5))
ax2 = ax1.twinx()
for mkt_col in ['qqq', 'spy']:
    ax1.plot(full_dataset['Date'], full_dataset[mkt_col], label=mkt_col)
ax2.plot(full_dataset['Date'], full_dataset['vxx'], label='vxx', c='r')
plt.title('Historical performance of indexes')
ax1.legend()
ax2.legend()
plt.show()

#### Appears that features 'spy' (S&P 500 index) and 'qqq' (NASDAQ 100 index) are correlated

Removing correlated features is important to prevent the model from assigning too much importance to, in essence, the same variable.

In [ ]:
sns.heatmap(full_dataset[features].corr())

In [ ]:
correlated_features = ['Enterprise Value Multiple', 'Quick Ratio (Acid Test)', 'qqq', 'Net Profit Margin', 
                       'Operating Profit Margin Before Depreciation']


In [ ]:
for i in range(len(correlated_features)):
    features.remove(correlated_features[i])
features_df = full_dataset[features]
target_df = full_dataset[target]

### Prepare data for modeling

#### Normal way to split data: random split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size=0.2, random_state=5)
print("Train set size: ", len(X_train), "\nTest set size: ", len(X_test))

#### Why is this a problem?

With time-dependent data, randomly splitting our train/test sets means we can have future data in the train set and past data in the test set.

In our case, this could mean teaching our model to predict the price of stock ABC on a given day using future data. 

This presents a data leakage problem. We want a time series split of our data.

In [ ]:
del X_train
del X_test
del y_train
del y_test

### Time Series data split

Lets save the latest 20% of data for testing

In [ ]:
cutoff_index = int(0.8 * len(features_df))

In [ ]:
X_train, X_test = features_df.iloc[:cutoff_index, :], features_df.iloc[cutoff_index:, :]
y_train, y_test = target_df.iloc[:cutoff_index], target_df.iloc[cutoff_index:]

### Normalizing data for training

We want to fit the normalizer only on the train set and apply that scaler to both. This prevents data leakage from train to test sets.

In [ ]:
ss = StandardScaler()
X_train = pd.DataFrame( # turning scaled output (np.Array) back into DataFrame
          ss.fit_transform(X_train),
          columns=X_train.columns,
          index=X_train.index
          )
            

In [ ]:
X_test = pd.DataFrame( # turning scaled output (np.Array) back into DataFrame
         ss.transform(X_test),
         columns=X_test.columns,
         index=X_test.index
         )

### Occam's Razor - can we solve this problem with a simpler model?

In [ ]:
class LinearRegressor:
    
    def __init__(self):
        self.w = None
        
    def fit(self, X, y):
       self.w = np.array(np.linalg.inv(X.T @ X) @ X.T @ y).reshape(-1,1)

    
    def predict(self, X):
        return (X @ self.w)[0]
    
    def get_weights(self):
        return self.w.round(4)
    

#### Fit Linear Regression 

In [ ]:
linreg = LinearRegressor()
linreg.fit(X_train, y_train)

In [ ]:
# out-of-sample prediction
lin_preds = linreg.predict(X_test)
np.mean(abs(lin_preds - y_test))

In [ ]:
full_dataset[['Ticker', 'Date']]

In [ ]:
full_dataset.iloc[cutoff_index:, :].loc[:, ['Ticker', 'Date']]

#### Spot checking Linear Regression predictions

In [ ]:
y_test_df = pd.DataFrame(y_test.round(3))

In [ ]:
compare_preds = pd.concat([full_dataset.iloc[cutoff_index:, :].loc[:, ['Ticker', 'Date']], 
                           pd.DataFrame(lin_preds.round(3)), 
                           y_test_df], 
                           axis=1)
compare_preds.columns = ['ticker', 'date', 'predicted price', 'true price']
compare_preds

#### Inspect linear regression weights

In [ ]:
pd.DataFrame(zip(features, linreg.get_weights()), columns=['feature', 'weight'])

### More powerful modeling

In [ ]:
params = {"n_estimators": 50,
          "max_depth": 5,
          "max_features": 0.7}

In [ ]:
rf = RandomForestRegressor(**params)
rf.fit(X_train, y_train)

In [ ]:
predictions_rf = rf.predict(X_test)

In [ ]:
# out-of-sample prediction
np.mean(abs(predictions_rf - y_test))

#### Spot checking Random Forest predictions

In [ ]:
compare_preds = pd.concat([full_dataset.iloc[cutoff_index:, :].loc[:, ['Ticker', 'Date']], 
                           pd.DataFrame(predictions_rf.round(3), index=y_test_df.index), 
                           y_test_df], 
                           axis=1)
compare_preds.columns = ['ticker', 'date', 'predicted price', 'true price']
compare_preds

### Explainable ML

#### Random Forest feature importance

Determines what features were most useful based on how often they were used to form decision trees in forest

In [ ]:
plt.barh(rf.feature_names_in_, rf.feature_importances_)

#### Random Forest Shapley Values

In [ ]:
# !! Warning: Takes a long time to run!! Better to read in Shapley from pickle below
explainer = shap.Explainer(rf.predict, X_test)
shap_values = explainer(X_test)

In [ ]:
# read in Shapley values from Pickle
shap_values = pickle.load(open('rf_shap.pkl', 'rb'))

In [ ]:
sample_ind = 100

#### Shap values are an intuitive way of representing the impact of features on individual predictions

As a quant in trading, your end users are the traders. It is imperative to translate quantitative solutions and insights to the business problem they are trying to solve.

In [ ]:
shap.plots.waterfall(shap_values[sample_ind])